# Transfer Learning

In [1]:
import sys

from tqdm.auto import tqdm
from tqdm.keras import TqdmCallback

sys.path.append('..')

import keras
import tensorflow as tf
import numpy as np

from src.TrainProdecure import train_single_model
from src.ModelBuilder import get_FCN
from src.LoadData import get_all_datasets_test_train_np_arrays, CurrentDatasets
from src.Ensemble import Ensemble, EnsembleMethods

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
EPOCHS_PER_TRAINING = 25
FINE_TUNE_DATASETS = CurrentDatasets.electric_devices.value, CurrentDatasets.cricket_y.value, CurrentDatasets.share_price_increase.value, CurrentDatasets.egg_five_days.value
EVALUATION_DATASET = CurrentDatasets.wafer
GENERATE_MODEL = get_FCN

In [3]:
ds = get_all_datasets_test_train_np_arrays('../datasets/')

In [4]:
def train_and_finetune(model,
                       x_pretrain: np.ndarray, y_pretrain: np.ndarray,
                       x_train_finetune: np.ndarray, y_train_finetune: np.ndarray,
                       fine_tuning_learning_rate, nr_epochs=EPOCHS_PER_TRAINING,
                       batch_size: int | None = 25, validation_split: float = 0.1,
                       model_name: str = 'Unnamed model',
                       pretrain_ds_name: str = 'Unnamed dataset', finetune_ds_name: str = 'Unnamed dataset',
                       optimizer=keras.optimizers.Adam):
    headless_model_layers = model.layers[:-1]

    # Pretraining:
    input_size = x_train.shape[1]
    output_size = len(np.unique(y_pretrain))
    pretrain_model = keras.Sequential([*headless_model_layers, keras.layers.Dense(output_size, activation='softmax')])
    pretrain_model.build(input_shape=(None, input_size, 1))
    train_single_model(pretrain_model, x_pretrain, y_pretrain, epochs=nr_epochs, model_name='pre-train'+model_name, dataset_name=pretrain_ds_name, batch_size=batch_size, validation_split=validation_split, optimizer=optimizer)

    # Fine Tuning:
    output_size = len(np.unique(y_train_finetune))
    input_size = x_train.shape[1]
    final_model = keras.Sequential([*headless_model_layers, keras.layers.Dense(output_size, activation='softmax')])
    final_model.build(input_shape=(None, input_size, 1))
    train_single_model(final_model, x_train_finetune, y_train_finetune, epochs=nr_epochs, model_name='fine-tune ' + model_name, dataset_name=finetune_ds_name, batch_size=batch_size, learning_rate=fine_tuning_learning_rate, validation_split=validation_split, optimizer=optimizer)

    return final_model

In [5]:
x_train_evaluation, y_train_evaluation = ds[EVALUATION_DATASET]['train_data']
x_test_evaluation, y_test_evaluation = ds[EVALUATION_DATASET]['test_data']

list_of_models = []

In [6]:
for dataset_name in tqdm(FINE_TUNE_DATASETS, desc='Training one model on several datasets', unit='Dataset'):
    x_train, y_train = ds[dataset_name]['train_data']
    x_test, y_test = ds[dataset_name]['test_data']

    input_size = x_train.shape[1]
    model = GENERATE_MODEL(input_size, 1)

    new_model = train_and_finetune(model, x_pretrain=x_train, y_pretrain=y_train,
                       x_train_finetune=x_train_evaluation, y_train_finetune=y_train_evaluation,
                       fine_tuning_learning_rate=10e-5, model_name='FCN',
                       pretrain_ds_name=dataset_name, finetune_ds_name=EVALUATION_DATASET)

    new_model.evaluate(x_test_evaluation, y_test_evaluation)

    list_of_models.append(new_model)

Training one model on several datasets:   0%|          | 0/4 [00:00<?, ?Dataset/s]

Training pre-trainFCN on electric_devices dataset: 0epoch [00:00, ?epoch/s]

Training fine-tune FCN on synthetic_control dataset: 0epoch [00:00, ?epoch/s]

10/10 [==============================] - 0s 5ms/step - loss: 0.5745 - accuracy: 0.8867


Training pre-trainFCN on cricket_y dataset: 0epoch [00:00, ?epoch/s]

...

Training fine-tune FCN on synthetic_control dataset: 0epoch [00:00, ?epoch/s]

10/10 [==============================] - 0s 3ms/step - loss: 0.3178 - accuracy: 0.9833


Training pre-trainFCN on egg_five_days dataset: 0epoch [00:00, ?epoch/s]

Training fine-tune FCN on synthetic_control dataset: 0epoch [00:00, ?epoch/s]

10/10 [==============================] - 0s 3ms/step - loss: 0.5389 - accuracy: 0.9500


## Build model from scratch

In [7]:
model_from_scratch = GENERATE_MODEL(x_train_evaluation.shape[1], len(np.unique(y_train_evaluation)))
model_from_scratch.build(input_shape=(None, x_train_evaluation.shape[1], 1))
model_from_scratch.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [11]:
model_from_scratch.fit(x_train_evaluation, y_train_evaluation, epochs=100, batch_size=25, validation_split=0.1, callbacks=[TqdmCallback(verbose=0, desc=f'Training from scratch on {EVALUATION_DATASET}')], verbose=0)

Training from scratch on synthetic_control: 0epoch [00:00, ?epoch/s]

In [12]:
model_from_scratch.evaluate(x_test_evaluation, y_test_evaluation)

10/10 [==============================] - 0s 24ms/step - loss: 0.0407 - accuracy: 0.9833


[0.04071700945496559, 0.9833333492279053]

## Build Ensemble from Pretrained models

In [13]:
ensemble = Ensemble(list_of_models, EnsembleMethods.AVERAGE)
ensemble.compile(metrics=['accuracy'])
ensemble.evaluate(x_test_evaluation, y_test_evaluation)

10/10 [==============================] - 1s 26ms/step - loss: 0.0000e+00 - accuracy: 0.1000


[0.0, 0.10000000149011612]